## Ejercicio Time Series Forecast
Para este ejercicio vamos a predecir cuál será la demanda de pasajeros de una aerolinea, para poder anticiparse a las contrataciones de personal, mantenimiento de las aeronaves y gestión de inventario y comidas.

Para ello, se pide:
1. Carga datos (AirPassengers.csv) y representa la serie. ¿Hay seasonality? ¿Cada cuanto?
2. Crea en una gráfica la variable original + su media obtenida mediante una rolling window con el valor de seasonality obtenido en el apartado anterior. Tienes que usar la función rolling() del DataFrame.
3. Comprueba de manera estadística si la serie es o no stationary.
4. Aplica una transformación logarítmica sobre los datos para mejorar el proceso de transformación de tu time series a stationary. Acuérdate después del forecast de invertir la transformación.
5. Divide en train y test. Guarda 20 muestras para test.
6. Crea tu primer modelo ARIMA. Habrá varias combinaciones en función de sus hiperparámetros... Mide el MAE y RMSE del modelo en predicción. Ten en cuenta el parámetro "m" de la función ARIMA, mediante el cual se establece el seasonality.
7. Representa en una gráfica los datos de test y tus predicciones.
8. Prueba un decission tree y un random forest, a ver qué performance presentan.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

## 1. Carga datos y representa la serie

In [ ]:
df = pd.read_csv('data/AirPassengers.csv')
df.head()

In [ ]:
df.info()

In [ ]:
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)
df.head()

In [ ]:
df.plot();

## 2. Crea en una gráfica la variable original + su media obtenida mediante una rolling window comparando el valor de seasonality obtenido en el apartado anterior

In [ ]:
df

In [ ]:
rolling = df.rolling(window=12).mean()

In [ ]:
plt.plot(df, color='blue', label='Original')
plt.plot(rolling.dropna(), color='red', label='Rolling Mean 12')
plt.legend()
plt.title("Rolling Mean")

## 3. Comprueba de manera estadística si la serie es o no stationary.

In [ ]:
from statsmodels.tsa.stattools import adfuller
adfuller(df['value'])[1]

## 4. Aplica una transformación logarítmica

In [ ]:
df['value'] = np.log(df['value'])

In [ ]:
df['value'].plot()

## 5. Divide en train y test. Guarda 20 muestras para test.

In [ ]:
len(df)

In [ ]:
train = df['value'][:124]
test = df['value'][124:]

In [ ]:
print(train.shape)
print(test.shape)

## 6. Crea tu primer modelo ARIMA

In [ ]:
#!pip install pmdarima

In [ ]:
from pmdarima.arima import auto_arima
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

model = auto_arima(
    train,
    start_p=1,
    start_q=1,
    max_d=3,
    max_p=5,
    max_q=5,
    stationary=False
    # trace=True
)

In [ ]:
model

In [ ]:
predictions = model.predict(20)
predictions

In [ ]:
test.values

In [ ]:
print("MAE", mean_absolute_error(test.values, predictions))
print("MAPE", mean_absolute_percentage_error(test.values, predictions))

In [ ]:
model = auto_arima(
    train,
    start_p=1,
    start_q=1,
    max_d=3,
    max_p=5,
    max_q=5,
    stationary=False,
    m=12
    # trace=True
)

In [ ]:
model

In [ ]:
predictions = model.predict(20)
print("MAE", mean_absolute_error(test.values, predictions))
print("MAPE", mean_absolute_percentage_error(test.values, predictions))

## 7. Representa en una gráfica los datos de test y tus predicciones.

In [ ]:
plt.plot(test.values, color="blue", label="Valores test")
plt.plot(predictions, color="red", label="Valores prediciones")
plt.legend();

## 8. Prueba otros modelos, a ver qué performance presentan.

In [ ]:
for i in range(12,0,-1):
    df['t-'+str(i)] = df['value'].shift(i)

df.dropna(inplace=True)
df

In [ ]:
X = df.iloc[:,1:]
y = df['value']

X_train = X[:112]
X_test = X[112:]

y_train = y[:112]
y_test = y[112:]

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rfr = RandomForestRegressor()
rfr.fit(X_train, y_train)

mean_absolute_percentage_error(y_test, rfr.predict(X_test))

In [ ]:
y_test

In [ ]:
X_test

In [ ]:
rfr.predict(X_test)


In [ ]:
plt.plot(y_test.values, color="blue", label="Valores test")
plt.plot(rfr.predict(X_test), color="red", label="Valores prediciones")
plt.legend();

In [ ]:
rfr.feature_importances_

In [ ]:
df['t-1'] = df['value'].shift(1)
df['t-11'] = df['value'].shift(11)
df['t-12'] = df['value'].shift(12)
df['t-13'] = df['value'].shift(13)
df

In [ ]:
df.dropna(inplace=True)
X = df.iloc[:,2:]
y = df['value']

X_train = X[:112]
X_test = X[112:]

y_train = y[:112]
y_test = y[112:]

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
rfr = RandomForestRegressor()
rfr.fit(X_train, y_train)

print(mean_absolute_percentage_error(y_test, rfr.predict(X_test)))
print(mean_absolute_percentage_error(y_train, rfr.predict(X_train)))

In [ ]:
plt.plot(y_test.values, color="blue", label="Valores test")
plt.plot(rfr.predict(X_test), color="red", label="Valores prediciones")
plt.legend();